<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Eduardo Moya
- Nombre de alumno 2: Nicolás Ojeda

### **Link de repositorio de GitHub:** [https://github.com/eduardomoyab/MDS7202_1](https://github.com/eduardomoyab/MDS7202_1)

## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `http://....`

# Importamos librerias útiles

In [100]:
%pip install -qq xgboost optuna

Note: you may need to restart the kernel to use updated packages.


# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [1]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import pickle

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

C:\Users\andmo\AppData\Local\Temp\ipykernel_3164\3184305967.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [3]:
# Paso 1: Separar los datos en conjuntos de train, validation y test
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
test_data, val_data = train_test_split(test_data, test_size=1/3, random_state=42)

# Paso 2: Implementar un FunctionTransformer para extraer el día, mes y año de la variable date
def extract_date_info(df):
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    df = df.drop(columns = ['date'])
    return df

date_transformer = FunctionTransformer(extract_date_info, validate=False)

# Paso 3: Implementar un ColumnTransformer para procesar los datos numéricos y categóricos
numeric_features = ['lat', 'long', 'pop','price']
categorical_features = ['city', 'shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='error',sparse_output=False), categorical_features)],
        remainder="passthrough",
        verbose_feature_names_out=False)
preprocessor.set_output(transform='pandas') #Lo fijamos como output pandas

# Paso 4: Guardar los pasos anteriores en un Pipeline con DummyRegressor
pipeline_dummy = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor())
])

# Paso 5: Entrenar el pipeline y reportar la métrica mean_absolute_error sobre los datos de validación
# Hacemos el split de X e Y para train, validation y testeo
global X_train, Y_train, X_val, Y_val, X_test, Y_test
X_train = train_data.drop('quantity', axis=1)
Y_train = train_data['quantity']
X_val = val_data.drop('quantity', axis=1)
Y_val = val_data['quantity']
X_test = test_data.drop('quantity', axis=1)
Y_test = test_data['quantity']

#Entrenamos el pipeline DummyRegressor
pipeline_dummy.fit(X_train, Y_train)
val_predictions_dummy = pipeline_dummy.predict(X_val)
mae_dummy = mean_absolute_error(Y_val, val_predictions_dummy)
#r_2_dummy = r2_score(Y_val, val_predictions_dummy)
print(f'MAE con Dummy Regressor: {mae_dummy}')


MAE con Dummy Regressor: 13413.17673026018


**Poner interpretación de esta métrica:** BLABLABLA

In [4]:
# Paso 6: Entrenar el Pipeline con XGBRegressor y reportar el cambio en MAE
pipeline_xgb = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

pipeline_xgb.fit(X_train, Y_train)
val_predictions_xgb = pipeline_xgb.predict(X_val)
mae_xgb = mean_absolute_error(Y_val, val_predictions_xgb)
#r_2_xgb = r2_score(Y_val, val_predictions_xgb)
print(f'MAE con XGBoost: {mae_xgb}')


MAE con XGBoost: 2390.7086816537158


¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?: BLABLABLA

En este caso al implementar el algoritmo XGBRegressor el MAE disminuye considerablemente de un valor del orden de 13000 a uno del orden de 2400. Si bien la métrica es poco mas de 10000 veces menor que con el Dummy Regressor, sigue siendo un valor alto considerando que se trata del contexto de negocios, por ende, pese a haber una mejora significativa, sigue siendo un mal desempeño.

In [5]:
# Paso 7: Guardar ambos modelos en un archivo .pkl
with open('regressor_dummy.pkl', 'wb') as f:
    pickle.dump(pipeline_dummy, f)
with open('regressor_xgb.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb, f)


## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [6]:
#Creamos un nuevo pipeline utilizando XGBRegressor con constraints
pipeline_xgb_inv = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(monotone_constraints={'price':-1})) 
])

pipeline_xgb_inv.fit(X_train, Y_train)
val_predictions_xgb_inv = pipeline_xgb_inv.predict(X_val)
mae_xgb_inv = mean_absolute_error(Y_val, val_predictions_xgb_inv)
#r_2_xgb_inv = r2_score(Y_val, val_predictions_xgb_inv)
print(f'MAE con XGBoost: {mae_xgb_inv}')
# Save the model with constraints to a .pkl file
with open('model_with_constraints.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb_inv, f)

MAE con XGBoost: 2529.758637824583


En este caso se puede observar que la métrica MAE aumentó en comparación con el XGBRegressor sin la relación del precio con la demanda, aunque de forma muy ligera. De acuerdo a lo que mencionaba el amigo, esta métrica debería haber disminuído, sin embargo ocurrió lo contrario, lo que hace suponer que la relación que existe entre la variable objetivo de demanda con el atributo precio no es linealmente inversa, por lo que se esta haciendo un supuesto que es falso y con ello empeorando el rendimiento.

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

**EXPLICAR CADA HIPERPARAMETRO**

            A

In [7]:
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
import joblib
from sklearn.preprocessing import FunctionTransformer
from xgboost import XGBRegressor
import joblib
import pandas as pd
import numpy as np

In [24]:
# Fijar la semilla
np.random.seed(42)
# Define the objective function
def objective(trial):
    # Dividir los datos

    # Definir hiperparámetros a optimizar para XGBRegressor
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
    }

    # Hiperparámetros para OneHotEncoder
    onehot_params = {
        'min_frequency': trial.suggest_float('min_frequency', 0.0, 1.0)
    }

    # Crear el pipeline con los hiperparámetros sugeridos
    pipeline_xgb = Pipeline([
        ('date_transformer', date_transformer),
        ('preprocessor', ColumnTransformer(
            transformers=[
                ('num', MinMaxScaler(), numeric_features),
                ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False, **onehot_params), categorical_features)
            ],
            remainder="passthrough",
            verbose_feature_names_out=False
        )),
        ('regressor', XGBRegressor(seed=42,**xgb_params))
    ])

    # Entrenar el modelo
    pipeline_xgb.fit(X_train, Y_train)

    # Realizar predicciones en el conjunto de validación
    val_predictions_xgb = pipeline_xgb.predict(X_val)

    # Calcular el MAE
    mae_xgb = mean_absolute_error(Y_val, val_predictions_xgb)

    return mae_xgb


In [25]:
# Configurar optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
sampler = TPESampler(seed=42)
study_opt = optuna.create_study(direction='minimize', sampler=sampler)

# Ejecutar la optimización
study_opt.optimize(objective, timeout=300, show_progress_bar=True)

# Obtener los mejores hiperparámetros
best_params = study_opt.best_params
best_mae = study_opt.best_value
num_trials = len(study_opt.trials)

print(f'Número de trials: {num_trials}')
print(f'MAE con los mejores hiperparámetros: {best_mae}')
print(f'Mejores hiperparámetros: {best_params}')

   0%|          | 00:00/05:00

Número de trials: 235
MAE con los mejores hiperparámetros: 2011.3882909370811
Mejores hiperparámetros: {'learning_rate': 0.062183929680474766, 'n_estimators': 798, 'max_depth': 8, 'max_leaves': 79, 'min_child_weight': 3, 'reg_alpha': 0.035506062538097426, 'reg_lambda': 0.04739167014450342, 'min_frequency': 0.03859764340935939}


In [26]:
# Obtener el mejor modelo con los mejores hiperparámetros
params_xgb = {
    'learning_rate': best_params['learning_rate'],
    'n_estimators': best_params['n_estimators'],
    'max_depth': best_params['max_depth'],
    'max_leaves': best_params['max_leaves'],
    'min_child_weight': best_params['min_child_weight'],
    'reg_alpha': best_params['reg_alpha'],
    'reg_lambda': best_params['reg_lambda'],
}
best_model = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', MinMaxScaler(), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False, min_frequency=best_params['min_frequency']), categorical_features)
        ],
        remainder="passthrough",
        verbose_feature_names_out=False
    )),
    ('regressor', XGBRegressor(**params_xgb))
])

# Entrenar el mejor modelo con todos los datos
best_model.fit(X_train, Y_train)
val_predictions_xgb_opt = best_model.predict(X_val)
mae_xgb_opt = mean_absolute_error(Y_val, val_predictions_xgb_opt)
print(f'MAE con XGBoost Optimizado: {mae_xgb_opt}')
print(f'Número de trials: {num_trials}')
#Mostramos los hiperparámetros del mejor modelo
print('')
print('Mejores hiperparámetros:')
#Imprimimos los hiperparámetros del modelo del diccionario best_params de la forma: hiperparámetro: valor
for key, value in best_params.items():
    print(f'{key}: {value}')
# Guardar el modelo en un archivo .pkl
joblib.dump(best_model, 'best_model_optimized.pkl')


MAE con XGBoost Optimizado: 2011.3882909370811
Número de trials: 235

Mejores hiperparámetros:
learning_rate: 0.062183929680474766
n_estimators: 798
max_depth: 8
max_leaves: 79
min_child_weight: 3
reg_alpha: 0.035506062538097426
reg_lambda: 0.04739167014450342
min_frequency: 0.03859764340935939


['best_model_optimized.pkl']

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [27]:
# Define the objective function with Prunning
optuna.logging.set_verbosity(optuna.logging.WARNING)
def objective_prunners(trial):
    # Definir hiperparámetros a optimizar para XGBRegressor
    xgb_params = {
        'eval_metric': mean_absolute_error,
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
    }

    # Hiperparámetros para OneHotEncoder
    onehot_params = {
        'min_frequency': trial.suggest_float('min_frequency', 0.0, 1.0)
    }

    # Configurar Prunning
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, observation_key="validation_0-mean_absolute_error")

    # Crear el pipeline con los hiperparámetros sugeridos y Prunning
    preprocessor_opt = ColumnTransformer(
        transformers=[
            ('num', MinMaxScaler(), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='error',sparse_output=False, **onehot_params), categorical_features),
            ])
    preprocessor_opt.set_output(transform='pandas') #Lo fijamos como output pandas

    pipeline_xgb = Pipeline([
        ('date_transformer', date_transformer),
        ('preprocessor',preprocessor_opt),
        ('regressor', XGBRegressor(seed=42,**xgb_params))
    ])

    pipeline_procesamiento = Pipeline(steps=[
        ('date_extraction', date_transformer),
        ('preprocessor', preprocessor_opt),
    ])


    # Entrenar el preprocesador con el conjunto de entrenamiento
    pipeline_procesamiento.fit_transform(X_train)

    # Aplicar la misma transformación al conjunto de validación
    X_val_processed = pipeline_procesamiento.transform(X_val)
    #X_val_processed= pipeline_procesamiento.fit_transform(X_val)


    # Entrenar el modelo con Prunning
    pipeline_xgb.fit(X_train, Y_train, regressor__callbacks=[pruning_callback], regressor__eval_set=[(X_val_processed,Y_val),])

    # Realizar predicciones en el conjunto de validación
    val_predictions_xgb = pipeline_xgb.predict(X_val)

    # Calcular el MAE
    mae_xgb = mean_absolute_error(Y_val, val_predictions_xgb)

    return mae_xgb

# Configurar optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)
sampler = TPESampler(seed=42)
study_opt_pr = optuna.create_study(direction='minimize', sampler=sampler)

# Ejecutar la optimización con Prunning
study_opt_pr.optimize(objective_prunners, timeout=300,show_progress_bar = True)

# Obtener los mejores hiperparámetros
best_params = study_opt_pr.best_params
best_mae = study_opt_pr.best_value
num_trials = len(study_opt_pr.trials)

print(f'Número de trials: {num_trials}')
print(f'MAE con los mejores hiperparámetros: {best_mae}')
print(f'Mejores hiperparámetros: {best_params}')

   0%|          | 00:00/05:00

[0]	validation_0-rmse:17057.97401	validation_0-mean_absolute_error:13144.96289
[1]	validation_0-rmse:16729.72126	validation_0-mean_absolute_error:12891.09961
[2]	validation_0-rmse:16417.78443	validation_0-mean_absolute_error:12650.47852
[3]	validation_0-rmse:16131.59562	validation_0-mean_absolute_error:12427.77637
[4]	validation_0-rmse:15855.93576	validation_0-mean_absolute_error:12212.74902
[5]	validation_0-rmse:15602.23880	validation_0-mean_absolute_error:12019.84473
[6]	validation_0-rmse:15356.04670	validation_0-mean_absolute_error:11828.89551
[7]	validation_0-rmse:15130.23206	validation_0-mean_absolute_error:11659.19531
[8]	validation_0-rmse:14917.64015	validation_0-mean_absolute_error:11502.79394
[9]	validation_0-rmse:14711.60851	validation_0-mean_absolute_error:11350.34863
[10]	validation_0-rmse:14520.02348	validation_0-mean_absolute_error:11209.56934
[11]	validation_0-rmse:14340.34286	validation_0-mean_absolute_error:11079.71680
[12]	validation_0-rmse:14171.95605	validation_0-me

c:\Users\andmo\.conda\envs\rapidsai\lib\site-packages\xgboost\sklearn.py:885: UserWarning:

`callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.



[36]	validation_0-rmse:12263.44558	validation_0-mean_absolute_error:9431.53125
[37]	validation_0-rmse:12239.89749	validation_0-mean_absolute_error:9408.99512
[38]	validation_0-rmse:12219.56080	validation_0-mean_absolute_error:9387.73633
[39]	validation_0-rmse:12196.51335	validation_0-mean_absolute_error:9363.87012
[40]	validation_0-rmse:12183.01370	validation_0-mean_absolute_error:9347.35059
[41]	validation_0-rmse:12166.40772	validation_0-mean_absolute_error:9327.79785
[42]	validation_0-rmse:12155.51418	validation_0-mean_absolute_error:9311.01367
[43]	validation_0-rmse:12140.29312	validation_0-mean_absolute_error:9293.01856
[44]	validation_0-rmse:12130.83556	validation_0-mean_absolute_error:9278.70215
[45]	validation_0-rmse:12118.79779	validation_0-mean_absolute_error:9262.79394
[46]	validation_0-rmse:12111.94291	validation_0-mean_absolute_error:9252.06348
[47]	validation_0-rmse:12103.41273	validation_0-mean_absolute_error:9242.72363
[48]	validation_0-rmse:12092.77113	validation_0-mean

c:\Users\andmo\.conda\envs\rapidsai\lib\site-packages\xgboost\sklearn.py:885: UserWarning:

`callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.



...

[0]	validation_0-rmse:16412.96250	validation_0-mean_absolute_error:12634.49316
[1]	validation_0-rmse:15528.89674	validation_0-mean_absolute_error:11951.45606
[2]	validation_0-rmse:14721.85311	validation_0-mean_absolute_error:11342.93262
[3]	validation_0-rmse:13983.96803	validation_0-mean_absolute_error:10768.93457
[4]	validation_0-rmse:13301.07092	validation_0-mean_absolute_error:10281.29492
[5]	validation_0-rmse:12705.63269	validation_0-mean_absolute_error:9826.82812
[6]	validation_0-rmse:12170.12117	validation_0-mean_absolute_error:9416.96094
[7]	validation_0-rmse:11630.68931	validation_0-mean_absolute_error:8989.61621
[8]	validation_0-rmse:11116.75670	validation_0-mean_absolute_error:8592.77832
[9]	validation_0-rmse:10657.69926	validation_0-mean_absolute_error:8229.80273
[10]	validation_0-rmse:10254.95944	validation_0-mean_absolute_error:7891.43311
[11]	validation_0-rmse:9862.07198	validation_0-mean_absolute_error:7589.78418
[12]	validation_0-rmse:9530.63130	validation_0-mean_absolu

c:\Users\andmo\.conda\envs\rapidsai\lib\site-packages\xgboost\sklearn.py:885: UserWarning:

`callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.



[30]	validation_0-rmse:5915.06316	validation_0-mean_absolute_error:4244.48828
[31]	validation_0-rmse:5794.80551	validation_0-mean_absolute_error:4138.03223
[32]	validation_0-rmse:5680.81318	validation_0-mean_absolute_error:4049.92017
[33]	validation_0-rmse:5605.30434	validation_0-mean_absolute_error:3981.27881
[34]	validation_0-rmse:5513.77176	validation_0-mean_absolute_error:3895.28613
[35]	validation_0-rmse:5439.44902	validation_0-mean_absolute_error:3832.32227
[36]	validation_0-rmse:5364.43978	validation_0-mean_absolute_error:3774.72192
[37]	validation_0-rmse:5289.21060	validation_0-mean_absolute_error:3708.34888
[38]	validation_0-rmse:5214.47890	validation_0-mean_absolute_error:3650.90356
[39]	validation_0-rmse:5159.05053	validation_0-mean_absolute_error:3596.78223
[40]	validation_0-rmse:5106.46653	validation_0-mean_absolute_error:3562.64673
[41]	validation_0-rmse:5019.32501	validation_0-mean_absolute_error:3493.65015
[42]	validation_0-rmse:4972.25458	validation_0-mean_absolute_err

c:\Users\andmo\.conda\envs\rapidsai\lib\site-packages\xgboost\sklearn.py:885: UserWarning:

`callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.



[31]	validation_0-rmse:5531.09328	validation_0-mean_absolute_error:3948.88208
[32]	validation_0-rmse:5435.01964	validation_0-mean_absolute_error:3852.36328
[33]	validation_0-rmse:5360.36792	validation_0-mean_absolute_error:3791.39941
[34]	validation_0-rmse:5272.90742	validation_0-mean_absolute_error:3708.52441
[35]	validation_0-rmse:5205.19679	validation_0-mean_absolute_error:3643.10693
[36]	validation_0-rmse:5143.57735	validation_0-mean_absolute_error:3576.70508
[37]	validation_0-rmse:5068.03103	validation_0-mean_absolute_error:3507.02417
[38]	validation_0-rmse:5000.42926	validation_0-mean_absolute_error:3444.04077
[39]	validation_0-rmse:4936.92412	validation_0-mean_absolute_error:3392.23804
[40]	validation_0-rmse:4875.35240	validation_0-mean_absolute_error:3341.08520
[41]	validation_0-rmse:4825.72873	validation_0-mean_absolute_error:3292.06177
[42]	validation_0-rmse:4774.63637	validation_0-mean_absolute_error:3246.25732
[43]	validation_0-rmse:4720.46775	validation_0-mean_absolute_err

In [19]:

# Obtener el mejor modelo con los mejores hiperparámetros
params_xgb_prunning = {
    'learning_rate': best_params['learning_rate'],
    'n_estimators': best_params['n_estimators'],
    'max_depth': best_params['max_depth'],
    'max_leaves': best_params['max_leaves'],
    'min_child_weight': best_params['min_child_weight'],
    'reg_alpha': best_params['reg_alpha'],
    'reg_lambda': best_params['reg_lambda'],
}


preprocessor_opt = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='error',sparse_output=False, min_frequency=best_params['min_frequency']), categorical_features),
        ])
best_model = Pipeline([
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor_opt),
    ('regressor', XGBRegressor(seed=42,**params_xgb_prunning))
])

# Entrenar el mejor modelo con todos los datos
best_model.fit(X_train, Y_train)
val_predictions_xgb_prunning = best_model.predict(X_val)
mae_xgb_prunning = mean_absolute_error(Y_val, val_predictions_xgb_prunning)
print(f'MAE con XGBoost Optimizado y Prunning: {mae_xgb_prunning}')
print(f'Número de trials: {num_trials}')
# Mostramos los hiperparámetros del mejor modelo
print('')
print('Mejores hiperparámetros:')
# Imprimimos los hiperparámetros del modelo del diccionario best_params de la forma: hiperparámetro: valor
for key, value in best_params.items():
    print(f'{key}: {value}')

# Guardar el modelo en un archivo .pkl
joblib.dump(best_model, 'best_model_optimized_with_pruning.pkl')

MAE con XGBoost Optimizado y Prunning: 1929.8149322427948
Número de trials: 184

Mejores hiperparámetros:
learning_rate: 0.09687712594172157
n_estimators: 831
max_depth: 9
max_leaves: 68
min_child_weight: 1
reg_alpha: 0.734771686412398
reg_lambda: 0.4356571909842588
min_frequency: 0.051201924629518156


['best_model_optimized_with_pruning.pkl']

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [35]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances


In [36]:
# Gráfico de historial de optimización con Optuna
plot_optimization_history(study_opt)

In [30]:
# Gráfico de historial de optimización con Optuna y Prunning
plot_optimization_history(study_opt_pr)

In [37]:
#Grafico de coordenadas paralelas con Optuna
plot_parallel_coordinate(study_opt)

In [38]:
#Grafico de coordenadas paralelas con Optuna y Prunning
plot_parallel_coordinate(study_opt_pr)

In [31]:
# Gráfico de importancia de hiperparámetros con Optuna
plot_param_importances(study_opt)

In [39]:
#Gráfico de importancia de hiperparámetros con Optuna y Prunning
plot_param_importances(study_opt_pr)

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento? 

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [43]:
#Creamos una tabla resumen con todos los MAE de los modelos
data = {'MAE': [mae_dummy, mae_xgb, mae_xgb_inv, mae_xgb_opt, mae_xgb_prunning]}
df_mae = pd.DataFrame(data, index =['Dummy Regressor', 'XGBoost', 'XGBoost Constraint', 'XGBoost Optuna', 'XGBoost Optuna + Prunning'])
df_mae

,MAE
Dummy Regressor,13413.176730
XGBoost,2390.708682
XGBoost Constraint,2529.758638
XGBoost Optuna,2011.388291
XGBoost Optuna + Prunning,1929.814932


In [44]:
#Cargamos el modelo de optuna + prunning y predecimos sobre el conjunto test
best_model = joblib.load('best_model_optimized_with_pruning.pkl')
test_predictions_xgb_prunning = best_model.predict(X_test)
mae_xgb_prunning = mean_absolute_error(Y_test, test_predictions_xgb_prunning)
print(f'MAE con XGBoost Optimizado y Prunning: {mae_xgb_prunning}')

MAE con XGBoost Optimizado y Prunning: 1983.2560417845616


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>